<a href="https://colab.research.google.com/github/ev1025/happy_dog_map/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 강아지 행복지도

## **로컬DB happydog에서 크롤링파일 가져오기**

In [ ]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pymysql
import pymysql.cursors

# mysql 연결자 생성
conn = pymysql.connect(host='127.0.0.1',   # db주소
                       user='root',        # db유저아이디
                       password='s10e534', # db비밀번호
                       db='happydog', # db이름
                       charset='utf8',     # 한글설정
                       cursorclass = pymysql.cursors.DictCursor)


cur = conn.cursor() # 커서 생성

In [ ]:
# 일반 크롤링, 강아지도 사이트 크롤링 데이터 합치기
sql = '''
select * 
from star_dog_map
union
select * 
from happy_dog_list
'''

In [ ]:
# 데이터 가져와서 map_list에 저장
cur.execute(sql)
map_list = cur.fetchall()
print(map_list[0])

{'상호명': '조용한저녁', '업종': '양식', '리뷰 수': 0, '별점': '4.83', '행정구': '마포구', '전화번호': '0507-1431-4540'}


## 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(map_list)

**중복제거**

In [ ]:
len(df)

1944

In [ ]:
df['상호명'].duplicated().sum()

34

In [ ]:
df.drop_duplicates(subset=['상호명'], ignore_index=True, inplace=True)

In [ ]:
len(df)

1910

**업종명이 불명확한 부분이 많아서 업종명을 정리**

In [ ]:
df.columns

Index(['상호명', '업종', '리뷰 수', '별점', '행정구', '전화번호'], dtype='object')

In [ ]:
df['업종'].unique()

array(['양식', '카페,디저트', '햄버거', '국밥', '카페', '도시락,컵밥', '한식', '바(BAR)', '죽',
       '다이어트,샐러드', '뜨개', '커피번', '아이스크림', '빙수', '베이글', '베이커리',
       '스파게티,파스타전문', '호두과자', '과일,주스전문점', '꽃집,꽃배달', '라이브카페', '도넛', '서점',
       '와인', '테이크아웃커피', '복합문화공간', '퓨전음식', '피자', '애견카페', '브런치', '샌드위치',
       '플라워카페', '맥주,호프', '태국음식', '케이크전문', '다방', '육류,고기요리', '떡볶이', '아시아음식',
       '스페인음식', '멕시코,남미음식', '주류', '이탈리아음식', '돼지고기구이', '일식당', '요리주점',
       '사진,스튜디오', '카레', '채식,샐러드뷔페', '테마카페', '반려동물미용', '해물,생선요리', '스테이크,립',
       '72420', '양갈비', '곱창,막창,양', '만두', '덮밥', '생선회', '프랑스음식', '돈가스',
       '포장마차', '일식튀김,꼬치', '닭갈비', '북카페', '종합분식', '찌개,전골', '이자카야', '인도음식',
       '미용실', '스포츠시설', '네일아트,네일샵', '반려견놀이터', '애견용품', '족발,보쌈', '조개요리',
       '치킨,닭강정', '주꾸미요리', '분식', '닭발', '베트남음식', '닭볶음탕', '소고기구이', '냉면',
       '백숙,삼계탕', '장어,먹장어요리', '갤러리카페', '차', '전통,민속주점', '우동,소바', '순대,순댓국',
       '중식당', '일본식라면', '술집', '오뎅,꼬치', '커피가공,제조', '장소대여', '스크린골프장',
       '반려동물용품', '반려동물호텔', '애견훈련', '반려동물', '반려동물분양', '약국', '동물병원', '애견사료',
      

In [ ]:
# 큰 카테고리로 분류
# 동반식당
df['업종'].replace(['양식','햄버거','국밥','도시락,컵밥','한식','일본식라면','바(BAR)','퓨전음식','죽','다이어트,샐러드','뜨개','와인','스파게티,파스타전문'
                 ,'스테이크,립','피자','브런치','샌드위치','맥주,호프','태국음식','이탈리아음식','육류,고기요리','떡볶이','아시아음식','스페인음식','멕시코,남미음식'
                 ,'주류','중식당','덮밥','요리주점','프랑스음식','일식당','돼지고기구이', '술집', '이자카야', '카레', '찌개,전골', '치킨,닭강정', '주꾸미요리', '분식', 
                  '닭발', '베트남음식', '닭볶음탕', '생선회', '곱창,막창,양', '소고기구이', '냉면', '백숙,삼계탕','전통,민속주점',  '포장마차', '게요리', '우동,소바', '토스트', '순대,순댓국'
                  , '오뎅,꼬치',  '닭갈비', '채식,샐러드뷔페', '해물,생선요리', '72420', '양갈비', '만두', '일식튀김,꼬치', '종합분식', '인도음식', '족발,보쌈', '조개요리','장어,먹장어요리'
                  , '돈가스'],'식당(동반)',inplace=True)
# 동반카페
df['업종'].replace(['카페,디저트','카페','커피번','아이스크림','빙수','베이글','베이커리','호두과자','과일,주스전문점','꽃집,꽃배달','라이브카페','도넛','북카페',
                  '테이크아웃커피','플라워카페','케이크전문','다방','갤러리카페', '차', '보드카페','예식장','커피가공,제조', '사주카페', '테마카페'],'카페(동반)', inplace=True)

# 애견미용
df['업종'].replace('반려동물미용','애견미용', inplace=True)

# 애견위탁
df['업종'].replace('반려동물호텔','애견호텔', inplace=True)

# 애견용품
df['업종'].replace(['반려동물분양','반려동물용품', '애견사료','애완동물', '애견의류'],'애견용품', inplace=True)

# 동물약국
df['업종'].replace('약국','동물약국', inplace=True)

# 간식, 놀이터
df['업종'].replace(['반려견놀이터','제조업','동물사료제조' ],'애견취미', inplace=True)

# 주인 취미
df['업종'].replace(['뜨개', '아기사진전문', '네일아트,네일샵', '셀프,대여스튜디오', '서점','복합문화공간','장소대여','사진,스튜디오',
                  '공방', '스크린골프장','스포츠시설',],'주인취미', inplace=True)

In [ ]:
df['업종'].unique()

array(['식당(동반)', '카페(동반)', '주인취미', '애견카페', '애견미용', '미용실', '애견취미', '애견용품',
       '애견호텔', '애견훈련', '반려동물', '동물약국', '동물병원', '직업,기술학원'], dtype=object)

In [ ]:
# # 이상치 확인용
# df[df['업종']=='인테리어소품']

In [ ]:
# 잘못 수집된 정보 제거
del_list = ['직업,기술학원','미용실','인테리어소품','편입학원']

del_list = '|'.join(del_list)
del_index = df[df['업종'].str.contains(del_list)].index

df.drop(index=del_index, inplace=True)

In [ ]:
df[df['업종']=='반려동물']

,상호명,업종,리뷰 수,별점,행정구,전화번호
1496,개러리아 송파점,반려동물,19,nan,송파구,0507-1420-3103
1497,프랑소와펫 신사점,반려동물,39,4.89,강남구,02-547-3362
1508,강아지유치원 퍼피스쿨,반려동물,11,nan,강남구,02-554-2010
1515,하울팟 케어클럽 서초본점,반려동물,109,nan,서초구,0507-1487-1108
1516,몰리스펫샵 성수점,반려동물,34,4,성동구,02-2104-5024
1527,리본,반려동물,16,nan,양천구,0507-1322-4021
1530,킨더가든포독,반려동물,4,nan,강남구,02-6032-5100
1701,금동네 사랑방,반려동물,20,4.72,송파구,0507-1349-8784
1705,견생식탁,반려동물,13,nan,동작구,0507-1350-8743
1707,빵댕잇,반려동물,14,nan,종로구,0507-1391-4670


In [ ]:
# 반려동물테마 분류용
# df[df['업종']=='반려동물'].iloc[[15,17]].index

In [ ]:
# 업종이 반려동물인 경우 세분화해서 넣어줌
df.loc[[1496,1701, 1705, 1707],'업종'] = '애견취미'
df.loc[[1497,1516],'업종'] = '애견용품'
df.loc[1768,'업종'] = '애견호텔'
df.loc[[1508, 1515, 1527, 1530, 1745, 1747, 1748, 1749, 1750, 1756, 1766, 1770],'업종'] = '애견훈련'
df.loc[[1751, 1761],'업종'] = '주인취미'

In [ ]:
df[df['업종']=='반려동물']

,상호명,업종,리뷰 수,별점,행정구,전화번호


In [ ]:
df['업종'].unique()

array(['식당(동반)', '카페(동반)', '주인취미', '애견카페', '애견미용', '애견취미', '애견용품', '애견호텔',
       '애견훈련', '동물약국', '동물병원'], dtype=object)

In [ ]:
# 별점이 없는(nan) 행들 0으로 변경, 타입을 float으로 변경
nan_index = df[df['별점'].str.contains('nan')]['별점'].index
df.loc[nan_index,'별점'] = df.loc[nan_index,'별점'].str.replace('nan', '0')
df['별점'] = df['별점'].astype(float)

In [ ]:
# 별점이 nan이 아닌 값의 평균을 구함
not_nan_index = [i for i in df.index.tolist() if i not in nan_index.tolist()]
star_nnul = df.loc[not_nan_index]
star_nnul = round(star_nnul.groupby('업종')[['별점']].mean(),2).to_dict()
star_dict = star_nnul['별점']
star_dict

{'동물병원': 4.56,
 '동물약국': 4.39,
 '식당(동반)': 4.57,
 '애견미용': 4.64,
 '애견용품': 4.54,
 '애견취미': 4.7,
 '애견카페': 4.58,
 '애견호텔': 4.57,
 '애견훈련': 4.65,
 '주인취미': 4.69,
 '카페(동반)': 4.58}

In [ ]:
# 해당 업종의 평균값으로 0인 값을 채워줌
for key, value in star_dict.items():
    rep_index = df[(df['업종']=='key')|df['별점']==0].index
    df.loc[rep_index,'별점'] = value

# 0인 가게 확인
df[df['별점']==0]

,상호명,업종,리뷰 수,별점,행정구,전화번호


In [ ]:
# df.to_csv(path_or_buf='../강아지도/last_dog_map.csv',encoding='utf-8-sig',index=False)

## 지표 만들기